In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from skimage.morphology import skeletonize
from skan import Skeleton, summarize
from tqdm import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path_manual = ''                 #Inserire percorso maschere manuali

path_pred = ''               #Inserire percorso maschere

set_pred = set(os.listdir(path_pred))                     #Prendiamo solo gli elementi di manual del test set
set_manual = set(os.listdir(path_manual))
set_pred_test = set_manual.intersection(set_pred)

list_pred = list(set_pred)
list_manual = list(set_manual)

In [ ]:
def dice_coefficient(imagepred, imagereal):
    image1 = imagepred > 0
    image2 = imagereal > 0
    intersection = sum(x & y for x, y in zip(image1.flatten(), image2.flatten()))
    total_pixels_image1 = sum(image1.flatten())
    total_pixels_image2 = sum(image2.flatten())

    dice = (2.0 * intersection) / (total_pixels_image1 + total_pixels_image2)

    return dice

def calcola_vasi_per_tipo(img):

  skt = skeletonize(img)

  skeleton_data = Skeleton(skt)
  skel_summary = summarize(skeleton_data)

  # count how many branches have branch-type = 1 and 2
  N_J2E = len(skel_summary[skel_summary['branch-type'] == 1])
  N_J2J = len(skel_summary[skel_summary['branch-type'] == 2])

  return N_J2E, N_J2J

def precision(image_pred, image_true):
    image1 = image_pred > 0
    image2 = image_true > 0
    true_positives = sum(x & y for x, y in zip(image1.flatten(), image2.flatten()))
    false_positives = sum(x & ~y for x, y in zip(image1.flatten(), image2.flatten()))

    if true_positives + false_positives == 0:
        return 0.0

    precision_value = true_positives / (true_positives + false_positives)
    return precision_value

def extract_subarrays_and_coordinates(arr, size):
    arrays = []
    coord_x = []
    coord_y = []

    # Ottieni le dimensioni dell'array in ingresso
    rows, cols = arr.shape

    # Estrai gli array quadrati di dimensione size
    for i in range(0, rows, size):
        for j in range(0, cols, size):
            subarray = arr[i:i+size, j:j+size]

            # Verifica le dimensioni dell'array estratto
            if subarray.shape == (size, size):
                arrays.append(subarray)
                coord_x.append(j)
                coord_y.append(i)
            else:
                # Stampa un messaggio di errore se le dimensioni non sono corrette
                print(f"Errore: Array di dimensioni {subarray.shape} invece di ({size}, {size})."
                      f" Ignorato a coordinate ({i}, {j}).")

    return arrays, coord_x, coord_y

def reconstruct_original_array(arrays, coord_x, coord_y):
    # Verifica che le liste siano della stessa lunghezza
    if len(arrays) != len(coord_x) or len(arrays) != len(coord_y):
        raise ValueError("Le liste di array e coordinate devono avere la stessa lunghezza.")

    # Ottieni la dimensione dell'array originale
    max_coord_x = max(coord_x)
    max_coord_y = max(coord_y)
    original_shape = (max_coord_y + arrays[0].shape[0], max_coord_x + arrays[0].shape[1])

    # Inizializza l'array originale con zeri
    original_array = np.zeros(original_shape)

    # Ricostruisci l'array originale
    for i in range(len(arrays)):
        x, y = coord_x[i], coord_y[i]
        original_array[y:y+arrays[i].shape[0], x:x+arrays[i].shape[1]] = arrays[i]

    return original_array

In [ ]:
metric_list = [[],[],[],[]]
for i in tqdm(range(len(list_pred))):
  dice = dice_coefficient(list_pred[i], list_manual[i])
  n_e_real, n_j_real = calcola_vasi_per_tipo( list_manual[i])
  n_e_pred, n_j_pred = calcola_vasi_per_tipo( list_pred[i])
  error_e = (n_e_real - n_e_pred)/n_e_real
  error_j = (n_j_real - n_j_pred)/n_j_real
  prec_score = precision(list_pred[i], list_manual[i])
  metric_list[0].append(dice)
  metric_list[1].append(error_e)
  metric_list[2].append(error_j)
  metric_list[3].append(prec_score)
dice_mean = np.mean(metric_list[0])
err_e_mean = np.mean(metric_list[1])
err_j_mean = np.mean(metric_list[2])
prec_mean = np.mean(metric_list[3])

print(f'la media del dice vale :{dice_mean}')
print(f'la media dell\' errore junction to endopoint vale :{err_e_mean}')
print(f'la media dell\' errore junction to junction vale : {err_j_mean}')
print(f'la media della precision vale : {prec_score}')